# Validating with MERIS datasets from 2010 and 2008 match-up time window of 2 hours

In [1]:
__credits__ = ["José M. Beltrán"]
__license__ = "GPL-3.0"
__created_on__ = ["2012-05-02"]
__modified_on__ = { "2016-01-07": ["Added Turbidity_Nechad_681"],
                    "2014-10-16": ["Batch process targeting Turbidity"], 
                    "2013-10-22": ["New GPT graph. The BEAM VISAT version 4.11 does not include anymore the inputPaths whithin the graph it has changed to: &lt;sourceProductPaths&gt;"], 
                    "2013-10-17": ["reintroduced for Selima and Evgeny: - xml_skeleton,- the get_fub_srcList "], 
                    "2012-06-18": ["xml_skeletons were replaced by gpt_skeletons", "introduced common.get_fub_srcList(srcDir) function", "introduced get_pins_srcList"]}

# **ipython-notebooks-notes** from the Turbidity Paper

*[2014-10-27]*
The first validation of retrieving SPM from Turbidity was done with: `[Method 4 Elina's] Turbidity and SPM validation-Validation_SPM_with_Matchups.ipynb`. This notebbok used the `nrrs` intermediate product of MEGS. However, note that:

`nrrs\_06 \times \pi` is almost `refletance\_06`, i.e., `refletance\_06 = \rho_{620}`

The differences may be due to the discretization of the decimals. An easy check is to plot the residuals between nrrs_06*pi and reflectance_06. It should be a gaussian distribution around 0. Which it means that are random errors due to precision on the discretisation.

See similar feature on ODESA forum [Differences on total\_susp and suspended\_particulate\_matter](http://www.odesa-info.eu/forum/viewtopic.php?f=16&t=165&sid=bf334be9585bbf3b8b67a25588c6e29e)  

Therefore a second validation was done using the `reflectance_06` values instead of `nrrs_06`. And the validation values were improved almost for 4%. See `[Method 4 Elina's] Turbidity and SPM validation-Validation_SPM_with_Matchups-using-reflectance_06`.

Results:
SPM\_Nechad\_Elinas using reflectance\_06: 3x3, Processor: MEGS
`MNB: -7.98865470193, RMS: 38.2198841474, NOBS: 32`

Using SPM\_Nechad\_Elinas using nrrs\_06: 3x3, Processor: MEGS 
`MNB: -11.8173847973, RMS: 36.7333234445, NOBS: 32`

__accompanying_paper__ : ["Kari et al Turbidity"]

__preceding_notebooks__ : ["create_placemarks_using_centroids", "pixel_extraction_centroids_MEGS-reflec_06", "[Method 4 Elina's] Turbidity and SPM validation"]
__anticipate_notebooks __ : []

In [1]:
import sys
import os.path
from sys import argv, exit
# Using the MathJax library to display the Latex.
from IPython.display import Math
#
import numpy as np
import pandas as pd

In [40]:
processor = "MEGS"
windowSize = "3"

In [3]:
os.chdir("/home/jobel/gits/jobel")

# Loading the pixel extraction files

In [4]:
# Creating the available pixel extraction files
srcDir = '/media/jobel/SeagateDrive/diana_meris/pixEx/'
pixEx_file_list = []
# Adding recursive walk for directories
for dirpath, dirnames, files in os.walk(srcDir):
    for file in files:  # files is a list of files in the current directory
        if file.endswith("_measurements.txt"):
            currentFilepath = dirpath
            pixEx_file_list.append(os.path.join(currentFilepath, file))   
pixEx_file_list[:3]
            

['/media/jobel/SeagateDrive/diana_meris/pixEx/pixEx_20090714_MER_FRS_2P_measurements.txt',
 '/media/jobel/SeagateDrive/diana_meris/pixEx/pixEx_20100605_MER_FRS_2P_measurements.txt',
 '/media/jobel/SeagateDrive/diana_meris/pixEx/pixEx_20100721_MER_FRS_2P_measurements.txt']

In [5]:
def get_MERIS_date_string_from_filename(filename):
    '''
    function to extract (substring) the name of the processor (FUB or MEGS), date and time
    from the filename of a MERIS processed dataset using regular expressions.
    returns list(processor,dayID,yyyy,mm,dd,hh,mm,ss)
    eg.
    filename: collocated_UTM33_WGS84_FUB_L1N_MER_FSG_CCL1P_20020808_101018_000003232008_00237_02295_0680.csv
    output:
    ["FUB",20020808,2002,08,08,10,10,18]
    '''
    import re

    regex = re.compile(r'(?P<year>[2][0][0-9][0-9])(?P<month>[0-1][0-9])(?P<day>[0-3][0-9])')
    x = regex.search(filename)
    all = x.group()
    date_string = all[:]  # dayID = to full date without time
    return date_string

In [21]:
#from files.regex_filename import get_MERIS_date_string_from_filename
# The pixEx files have a 6 head lines in the file that must be removed.
head_lines = 6 
pix_data = {}

for nf,filename in enumerate(pixEx_file_list):
    current_file = filename
    # Get the date value from the PixEX filename
    date_string = get_MERIS_date_string_from_filename(current_file)
    with open(current_file, 'r') as fr:
        #skipping headlines
        for i in xrange(head_lines):
            fr.next()
        column_headers = fr.next().strip('\n').split('\t') # Gets the column headers
        column_headers[7:9] = ['Date','Time'] # Changing header names to have only non special characters
        # The Date and Time fields are not reliable, therefore we must add our own column with a date_string
        column_headers.append("meris_date")
        pix_data[date_string] = dict([(key,[]) for key in column_headers])        
              
        for nr,  row_line in enumerate(fr):
            for column, value in enumerate(row_line.strip('\n').split('\t')):
                if column_headers[column] == 'reflectance_06':
                    if value != "":
                        pix_data[date_string][column_headers[column]].append(float(value))
                    else:
                        pix_data[date_string][column_headers[column]].append(np.nan)
                else:
                    pix_data[date_string][column_headers[column]].append(value)
            # Appending the date_string to keep track of the source file
            pix_data[date_string]["meris_date"].append(date_string)
            
            
        


In [22]:
pix_data.keys()

['20100831',
 '20100906',
 '20100811',
 '20100824',
 '20100605',
 '20100721',
 '20090714']

In [25]:
sorted(pix_data['20100906'].keys())[:3]

['CoordID', 'Date', 'Latitude']

In [26]:
# Merging the dictionaries as panda dataframes
# Using MEGS the reflectance_06 value is for Case-1 waters. The bands "nrrs_06", "c2r_RLw_06" comes from the intermediate products
# of the Case-2 branch, and those were u
keep_columns = ["meris_date","Name","CoordID",'l2_flags',"reflec_6","total_susp"]

meris = pd.DataFrame(columns=[keep_columns])
for key in pix_data.keys():
    meris = pd.merge(meris, pd.DataFrame(pix_data[key])[keep_columns],how='outer' )


# Renaming Name column to match insitu column name as ID_CAST
meris.rename(columns = {'Name':'ID_CAST'}, inplace=True)
meris.rename(columns = {'total_susp':'SPM'}, inplace=True)
#
meris[:3] # showing only the first three elements
# Note: dataHolder was used intead of meris in previous versions.

,meris_date,ID_CAST,CoordID,l2_flags,reflec_6,SPM
0,20100831,D17,1,8,0.002465893980115652,0.48421627283096313
1,20100831,D17,1,8,0.0027771827299147844,0.45034289360046387
2,20100831,D17,1,8,0.0028870494570583105,0.5597981810569763


In [14]:
# releasing memory
pix_data = None

## Note that $reflect\_6 = \rho_{620}$

### Calculating rho_w from Rrs

In [31]:
# Used to get a binary representation of the flags
def flag_as_binary(l2_flags):windowSize
    return bin(int(l2_flags))[2:]

In [33]:
def which_megs_l2_flags_N1(pixel_value):
    """
    the pixel_value corresponds to l2_flags using N1 file standard flags.
    return: a flag dictionary with ["raised_flags", "watch", "flagON", "pixel_value"]
    """
    # TIP: Use bin(value)[2:] to check the binary representation
    flags_dict = {}
    raised_flags = []
    watch = []
    # pixel_value type is numpy.float32 the function requires an integer
    # Catch  nan values
    #
    pixel_value = int(pixel_value)
    # Is the bit 23 on?
    if (pixel_value & (1 << 1)):  #
        watch.append("LOW_SUN")
    if (pixel_value & (1 << 5)):  #
        raised_flags.append("ICE_HAZE")
    if (pixel_value & (1 << 11)):  #
        raised_flags.append("SUSPECT")
    if (pixel_value & (1 << 22)):  #
        raised_flags.append("CLOUD")
    if (pixel_value & (1 << 23)):  #
        raised_flags.append("LAND")

    # Watch flags
    if (pixel_value & (1 << 0)):  #pickle
        watch.append("WHITE_SCATTERER")
    if (pixel_value & (1 << 2)):  #
        watch.append("HIGH_GLINT")
    if (pixel_value & (1 << 3)):  #
        watch.append("BPAC_ON")
    if (pixel_value & (1 << 4)):  #
        watch.append("MEDIUM_GLINT")
    if (pixel_value & (1 << 6)):  #
        watch.append("CASE2_Y")
    if (pixel_value & (1 << 7)):  #
        watch.append("CASE2_ANOM")
    if (pixel_value & (1 << 8)):  #
        watch.append("CASE2_S")
    if (pixel_value & (1 << 12)):  #
        watch.append("COSMETIC")
    if (pixel_value & (1 << 13)):  #
        watch.append("COASTLINE")
    if (pixel_value & (1 << 14)):  #
        watch.append("PCD_19")
    if (pixel_value & (1 << 16)):  #
        watch.append("PCD_17")
    if (pixel_value & (1 << 17)):  #
        watch.append("PCD_16")
    if (pixel_value & (1 << 21)):  #
        watch.append("WATER")

    #other flags
    if (pixel_value & (1 << 9)):  # 8192
        other.append("ABSOA_DUST")
    if (pixel_value & (1 << 10)):  # 8192
        other.append("O_AERO_DB")
    if (pixel_value & (1 << 15)):  # 8192
        other.append("PCD_18")
    if (pixel_value & (1 << 18)):  # 8192
        other.append("PCD_15")
    if (pixel_value & (1 << 19)):  # 8192
        other.append("PCD_14")
    if (pixel_value & (1 << 20)):  # 8192
        other.append("PCD_1_13")

    # checking if the length of the list to see if a flag was raised
    if len(raised_flags) >= 1:
        flagON = True
    else:
        flagON = False

    flags_dict["raised_flags"] = raised_flags
    flags_dict["watch"] = watch
    flags_dict["flagON"] = flagON
    flags_dict["pixel_value"] = pixel_value

    return flags_dict

In [34]:
# Importing function for quality check of the pixel values
#from geotools.meris import which_megs_l2_flags_N1

meris["flagON"] = [which_megs_l2_flags_NetCDF(value)['flagON'] for value in meris["l2_flags"]]
meris['raised_flags'] = [str(which_megs_l2_flags_NetCDF(value)['raised_flags']) for value in meris["l2_flags"]]
meris["watch"] = [str(which_megs_l2_flags_NetCDF(value)['watch']) for value in meris["l2_flags"]]

In [43]:
# The values of nrrs_06 are strings, so we need to convert those into float values.
# See that we still need to use the band nrrs_06 which does not have values where is crap data. reflectance does have.
for i, value in enumerate(meris['reflec_6']):
    if value == "":
        meris['reflec_6'][i] = np.nan        
    else:
        meris['reflec_6'][i] = np.float(meris['reflec_6'][i])
       
        
#meris['rho_w_06'] = map(rho_w,meris['reflectance_06'])
# See above to show why rho_w = reflectance_06
meris['rho_w_06'] = meris['reflec_6']
#meris['l2_flags_binary'] = map(flag_as_binary,meris['l2_flags'])
# showing the first three rows
meris[:10]

/home/jobel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,meris_date,ID_CAST,CoordID,l2_flags,reflec_6,SPM,flagON,raised_flags,watch,rho_w_06
0,20100831,D17,1,8,0.002465894,0.48421627283096313,False,[],[],0.002465894
1,20100831,D17,1,8,0.002777183,0.45034289360046387,False,[],[],0.002777183
2,20100831,D17,1,8,0.002887049,0.5597981810569763,False,[],[],0.002887049
3,20100831,D17,1,264,0.003308205,0.4669725298881531,False,[],[],0.003308205
4,20100831,D17,1,8,0.002539138,0.4669725298881531,False,[],[],0.002539138
5,20100831,D17,1,8,0.00156865,0.5020967125892639,False,[],[],0.00156865
6,20100831,D17,1,17408,0.001055939,0.4669725298881531,False,[],['OOADB'],0.001055939
7,20100831,D17,1,8,0.00189825,0.45034289360046387,False,[],[],0.00189825
8,20100831,D17,1,8,0.002520827,0.48421627283096313,False,[],[],0.002520827
9,20100831,D18,2,16384,0.001550339,0.6471778154373169,False,[],[],0.001550339


In [41]:
# Saving the meris dataframe for validation
outputfile= "/home/jobel/Dropbox/ElinasPaper/data_4_elina/pixEx_4_validation/meris_pixel_flags_"+ windowSize +'x'+windowSize +'_'+processor +'_matchups_radiometry_DIANAS_2016.txt'
meris.to_csv(path_or_buf=outputfile , sep="\t")

# Aggregating the data

In [51]:
# Initializing keys of the data dictionary (dd)
meris_dict = dict([((meris['meris_date'][i],meris['ID_CAST'][i]),{'rho_w_06':[],'SPM':[]}) for i, value in enumerate(meris['meris_date']) if meris['rho_w_06'][i] >0])

for i, value in enumerate(meris['meris_date']):    
    if (meris['rho_w_06'][i] >0) and (len(meris['SPM'][i])>0):
        try:
            meris_dict[(meris['meris_date'][i],meris['ID_CAST'][i])]['rho_w_06'].append(meris['rho_w_06'][i])              
            meris_dict[(meris['meris_date'][i],meris['ID_CAST'][i])]['SPM'].append(float(meris['SPM'][i]))
        except:
            print i, meris['meris_date'][i], meris['ID_CAST'][i], meris['SPM'][i], meris['SPM'][i]
            
#  
#dd[].append(value)
meris_dict.items()[:3]

[(('20100811', 'Kj6'),
  {'SPM': [7.085878372192383,
    7.9001946449279785,
    8.191922187805176,
    7.618855953216553,
    8.191922187805176,
    7.9001946449279785,
    7.618855953216553,
    8.191922187805176,
    7.9001946449279785],
   'rho_w_06': [0.007281715050339699,
    0.00482802651822567,
    0.008142337203025818,
    0.009387492202222347,
    0.008508559316396713,
    0.00735495937988162,
    0.007831048220396042,
    0.009442425332963467,
    0.0079958476126194]}),
 (('20100824', '12'),
  {'SPM': [27.106338500976562,
    26.14103889465332,
    26.14103889465332,
    27.106338500976562,
    27.106338500976562,
    28.107284545898438,
    27.106338500976562,
    27.106338500976562,
    27.106338500976562],
   'rho_w_06': [0.02002624422311783,
    0.003601182484999299,
    0.0246406439691782,
    0.039289526641368866,
    0.024201177060604095,
    0.022699665278196335,
    0.007318336982280016,
    0.015009001828730106,
    0.00740989251062274]}),
 (('20100721', 'Ra13'),
 

## Note how we have reflectance values where we have not in nrrs_06 (i.e. rho_w_06). 
## Therefore, as the rho_w_06 nan values correspond to off data of SPM. We can use the rho_w_06 to derive a model to retrieve Turbidity from MERIS MEGS.

### TODO: Check which flags were raised, for those stations, and use the recurrent flag to raise the flag while checking the quality of the pixels.

In [52]:
results = {}
results['rho_w_06'] = dict([((key,{"mean": np.mean(meris_dict[key]['rho_w_06']), "nobs":len(meris_dict[key]['rho_w_06']), "std":np.std(meris_dict[key]['rho_w_06'])})) for key in meris_dict.keys()])
results['SPM'] = dict([((key,{"mean": np.mean(meris_dict[key]['SPM']), "nobs":len(meris_dict[key]['SPM']), "std":np.std(meris_dict[key]['SPM'])})) for key in meris_dict.keys()])
results['SPM'].items()[:3]

/home/jobel/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/jobel/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


[(('20100811', 'Kj6'),
  {'mean': 7.8444378640916614, 'nobs': 9, 'std': 0.34136155828717335}),
 (('20100824', '12'),
  {'mean': 27.003043704562717, 'nobs': 9, 'std': 0.55472428687940267}),
 (('20100721', 'Ra13'),
  {'mean': 1.2452096276813083, 'nobs': 9, 'std': 0.07271988600473718})]

# Calculating tubidity based on Nechad et al 2009.

#TODO: Add the latex equation for turbidity algorithm

In [53]:
def turbidity_620(rho_w_06):
    # According with Nechad et al 2009.
    # Used as well within the coastcolour level 2 products.
    return ((174.41 *  rho_w_06) / (1-(rho_w_06 / 0.1533))) + 0.39

# Using Elinas Algorithm to get the SPM from turbidity
Use the [IPython's Rich Display System](http://nbviewer.ipython.org/github/ipython/ipython/blob/1.x/examples/notebooks/Part%205%20-%20Rich%20Display%20System.ipynb) to know more about how to add Math, Images, Videos, etc.. to an ipython notebook.


In [41]:
Math(r'\text{SPM} = \text{turbidity}_{swe} \times 0.9218 - 0.0422') # Previoulsy was 0.0416

<IPython.core.display.Math object>

In [62]:
# using the log transformed algorithm 
def calculate_spm_from_turbidity(turbidity):
    ln_SPM = 0.97 * np.log(turbidity) - 0.081
    return np.exp(ln_SPM)
    #return 0.9218 * turbidity - 0.0422
    # They gave the same results log or not log

# Joining with results

In [63]:
# joining the dictionaries, meris_aggregate will have the key of 'TURBIDITY_AVG_INSITU' with is value
results["joined_data"] = {}
for k in sorted(results['rho_w_06'].keys()):
    results["joined_data"][k] = {"SPM_meris":results['SPM'][k]["mean"],
                                 'rho_w_06':results['rho_w_06'][k]["mean"],                                     
                                 'turbidity_620':turbidity_620(results['rho_w_06'][k]["mean"]),                                     
                                 'SPM_Nechad_Elinas_620':calculate_spm_from_turbidity(turbidity_620(results['rho_w_06'][k]["mean"]))
                                 }
results["joined_data"]

#"SPM_Elinas":calculate_spm_from_turbidity(insitu_dict[k]["SPM"]),
#"SPM_Nechad_Elinas":calculate_spm_from_turbidity(turbidity_620(results['rho_w_06'][k]["mean"]))

{('20090714', '1'): {'SPM_Nechad_Elinas_620': 2.2439358020780102,
  'SPM_meris': 7.8596765200297041,
  'rho_w_06': 0.011114839774866899,
  'turbidity_620': 2.4800778933369605},
 ('20090714', '10'): {'SPM_Nechad_Elinas_620': 2.4829550877336155,
  'SPM_meris': 10.11720413631863,
  'rho_w_06': 0.012382375283373727,
  'turbidity_620': 2.739374145946643},
 ('20090714', '12'): {'SPM_Nechad_Elinas_620': 1.805646773192161,
  'SPM_meris': 7.0027636951870385,
  'rho_w_06': 0.0087303268826670125,
  'turbidity_620': 2.0046070440357573},
 ('20090714', '3B'): {'SPM_Nechad_Elinas_620': 1.93724238170386,
  'SPM_meris': 9.4981824556986485,
  'rho_w_06': 0.0094546328764408827,
  'turbidity_620': 2.1473664370838144},
 ('20090714', '5'): {'SPM_Nechad_Elinas_620': 5.458624611462688,
  'SPM_meris': 16.057195451524521,
  'rho_w_06': 0.026459546759724617,
  'turbidity_620': 5.9674816787401701},
 ('20100605', '13'): {'SPM_Nechad_Elinas_620': 0.75406167674750257,
  'SPM_meris': 2.0997011793984308,
  'rho_w_06':

## Sanity check

In [58]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.lines import Line2D   

In [64]:
# converting to Pandas Dataframe for easy visualization and easy export of the table
results_df = pd.DataFrame(results["joined_data"])
results_df 

20090714                                            \
                              1         10        12        3B          5   
SPM_Nechad_Elinas_620  2.243936   2.482955  1.805647  1.937242   5.458625   
SPM_meris              7.859677  10.117204  7.002764  9.498182  16.057195   
rho_w_06               0.011115   0.012382  0.008730  0.009455   0.026460   
turbidity_620          2.480078   2.739374  2.004607  2.147366   5.967482   

                       20100605                                          \
                             13         6        J4        J6        J7   
SPM_Nechad_Elinas_620  0.754062  0.616055  2.698750  0.413422  1.758653   
SPM_meris              2.099701  1.000000  3.082445  0.966092  5.219081   
rho_w_06               0.002669  0.001836  0.013507  0.000596  0.008470   
turbidity_620          0.863812  0.714097  2.973475  0.494274  1.953627   

                         ...      20100906                                \
                         ...          Rb16      Rb17       Rb2       Rb3   
SPM_Nechad_Elinas_620    ...      2.512408  0.408902  2.194218  2.061166   
SPM_meris                ...     12.223131  1.069217  6.619151  6.380124   
rho_w_06                 ...      0.012537  0.000568  0.010848  0.010130   
turbidity_620            ...      2.771325  0.489371  2.426142  2.281803   

                                                                         \
                            Rb4       Rb5       Rb6       Rb7       Rb8   
SPM_Nechad_Elinas_620  1.305631  1.170465  1.064813  1.249843  1.213605   
SPM_meris              2.719768  1.639022  1.211430  1.971169  1.531129   
rho_w_06               0.005910  0.005129  0.004513  0.005589  0.005379   
turbidity_620          1.462173  1.315541  1.200926  1.401652  1.362340   

                                 
                            Rb9  
SPM_Nechad_Elinas_620  0.981748  
SPM_meris              1.165064  
rho_w_06               0.004024  
turbidity_620          1.110813  

[4 rows x 67 columns]

In [61]:
filename = "/home/jobel/Dropbox/ElinasPaper/data_4_elina/Results/MERIS_SPM_vs_Insitu_matchups_"+ windowSize +'x'+windowSize +'_'+processor +'_DIANAS_data_update_2016.txt'
results_df.to_csv(filename, sep="\t")

# Calculating the Mean Normalized Bias and The RMS of the Relative Differences

In [28]:
Math(r'\textbf{MNB}= \textbf{mean} \left[\frac{y_{i}^{MERIS}- x_{i}^{insitu}}{x_{i}^{insitu}} \right] \times 100')

In [29]:
Math(r'\textbf{RMS}_{RD}= \textbf{stdev} \left[\frac{y_{i}^{MERIS}- x_{i}^{insitu}}{x_{i}^{insitu}} \right] \times 100')

## SPM insitu vs SPM calculated via Nechad-Elina

In [55]:
RD = []
for key in results["joined_data"].keys():
    insitu = results["joined_data"][key]['SPM_insitu']
    Meris = results["joined_data"][key]["SPM_Nechad_Elinas_620"]
    RD.append( (Meris - insitu) / insitu)
MNB = np.array(RD).mean() * 100
RMS = np.array(RD).std() * 100
NOBS = len(RD)
print ("SPM modelled: "+ windowSize +'x'+windowSize +', Processor: '+ processor+ ", MNB: "+ str(MNB)+ ", RMS: "+ str(RMS)+ ", NOBS: "+ str(NOBS))

SPM modelled: 3x3, Processor: MEGS, MNB: -8.04282185732, RMS: 38.19462919, NOBS: 32


In [56]:
RD = []
for key in results["joined_data"].keys():
    insitu = results["joined_data"][key]['SPM_insitu']
    Meris = results["joined_data"][key]["SPM_meris"]
    RD.append( (Meris - insitu) / insitu)
MNB = np.array(RD).mean() * 100
RMS = np.array(RD).std() * 100
NOBS = len(RD)
print ("SPM_MERIS: "+ windowSize +'x'+windowSize +', Processor: '+ processor+ ", MNB: "+ str(MNB)+ ", RMS: "+ str(RMS)+ ", NOBS: "+ str(NOBS))

SPM_MERIS: 3x3, Processor: MEGS, MNB: 11.6613109158, RMS: 46.0152141041, NOBS: 32


In [57]:
RD = []
for key in results["joined_data"].keys():
    insitu = results["joined_data"][key]['SPM_insitu']
    Meris = results["joined_data"][key]["SPM_Nechad_Elinas_680"]
    RD.append( (Meris - insitu) / insitu)
MNB = np.array(RD).mean() * 100
RMS = np.array(RD).std() * 100
NOBS = len(RD)
print ("SPM modelled: "+ windowSize +'x'+windowSize +', Processor: '+ processor+ ", MNB: "+ str(MNB)+ ", RMS: "+ str(RMS)+ ", NOBS: "+ str(NOBS))

SPM modelled: 3x3, Processor: MEGS, MNB: -27.3508111978, RMS: 32.6961505414, NOBS: 32
